## Neural Networks Homework 1 : Mustafa Nazlıer 15050111035


## Q1) Creating a dataset  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random


#Önce lineer olarak bir doğru yardımıyla ayırabileceğim noktalar generate edip artından noise ekleyeceğim
xA,yA,xB,yB = 1 ,2 ,4 ,3 #[random.uniform(0, 5) for i in range(4)]  #0 ile 5 arasında uniform skalar değerler random ediyorum
Line = np.array([xB*yA-xA*yB, yB-yA, xA-xB])    #düzlemi ortaya yakın bir yerden bölecek doğru denklemi 
        
X = []
S = []
for i in range(95):   #Kaç nokta
  x1,x2 = [random.uniform(0, 5) for i in range(2)]
  x = np.array([1,x1,x2])
  s = int(np.sign(Line.T.dot(x))) #s değeri 1 ise grafiğin altında -1 ise grafiğin üstündeolacak şekilde ayarladım
  X.append((x1,x2,s))
  S.append(s)

##Noise ekleme
for i in range(5): 
  x1,x2 = [random.uniform(0, 5) for i in range(2)]
  x = np.array([1,x1,x2])
  s = int(-np.sign(Line.T.dot(x))) #ters olacak şekilde 5 nokta ekliyorum 
  X.append((x1,x2, s))  #Tüm noktaları ve classifierlarını X arrayinde topluyorum grafiğe çizdikten sonra s değerlerine bu arrayde ihtiyacım kalmayacak
  S.append(s)        #Tüm classifierları ayrı olarak S te topluyorum       

    
fig = plt.figure(figsize=(5,5))


cols = {1: 'r', -1: 'b'}
for x in X:
    plt.plot(x[0], x[1], cols[x[2]]+'o')  #Noktalarıma Noise eklenmiş hali
    


## Q2) Adaline Network Sequential Mode

In [ ]:
Xdf = pd.DataFrame(X ,  columns=['x1', 'x2' ,'y'])     # X Arrayimi sadece koordinatlar kalacak şekilde düzenliyorum 
X_Global = Xdf.iloc[0:100,[0,1]].values  #Tüm sorularda kullanmak üzere bir X değeri oluşturuyorum

In [ ]:

learn_rate = 0.01
iterations = 500
X=np.array(X_Global)
X = np.c_[np.ones((X.shape[0], 1)), X]
y=np.array(S)
# = (X - np.mean(X, axis=0)) / np.std(X, axis = 0)
# random_gen = np.random.RandomState(1)

# weights = random_gen.normal(loc = 0.1, scale = 1, size= X.shape[0])
weights = np.random.uniform(-5, 5, X.shape[1] )
accuracyBefore = np.sum(y == np.where(np.dot(X, weights) >= 0.0, 1, -1)) / len(y)

print('Accuracy Before Training is '  , accuracyBefore)

costs = []

for cycle in range(iterations):
    error=0.0
    for i in range(X.shape[0]):
   
        output_pred = np.dot(X[i], weights) 
        y_pred = np.where(output_pred>=0, 1, -1)      #threshholding
        weights += (learn_rate * X[i]*(y[i] - y_pred))   #Sequential
        error += ((y[i] - y_pred) ** 2) / 2.0
        
    costs.append(error)


fig = plt.figure(figsize=(10,8))
plt.plot(range(1, len(costs) + 1), costs)
plt.title("Adaline:")
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.show()

accuracySequential = np.sum(y == np.where(np.dot(X, weights) >= 0.0, 1, -1)) / len(y)
print('Accuracy After Training is '  , accuracySequential)

## Q3) Adaline Network Batch Mode

In [ ]:
learn_rate = 0.01
iterations = 20
X=np.array(X_Global)
y= np.array(S)
   

weights = np.random.uniform(-5, 5, X.shape[1] )
accuracyBefore = np.sum(y == np.where(np.dot(X, weights) >= 0.0, 1, -1)) / len(y)
print('Accuracy Before Training is '  , accuracyBefore)


costs = []

for cycle in range(iterations):
       
      
        output_pred = np.dot(X, weights)
        errors = y - output_pred   
        weights += (learn_rate * X.T.dot(errors))/iterations
        cost = (errors**2).sum() / 2.0
        costs.append(cost)
   



plt.plot(range(1, len(costs) + 1), costs)
plt.title("Adaline")
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

accuracyBatch = np.sum(y == np.where(np.dot(X, weights) >= 0.0, 1, -1)) / len(y)
print('Accuracy After Training is '  , accuracyBatch)


## Q4) Adaline Network Mini-Batch Mode


In [ ]:
Batch_Size=10         #Mini-Batch sizeım

BatchDown=0           #Mini-Batch subseti için array indexi verebilmek için değerlerim
BatchUp=Batch_Size

learn_rate = 0.01
iterations = 100
                     
X=np.array(X_Global)
y= np.array(S)
   


weights = np.random.uniform(-5, 5, X.shape[1] ) #Weight randomize ediyorum

accuracyBefore = np.sum(y == np.where(np.dot(X, weights) >= 0.0, 1, -1)) / len(y)
print('Accuracy Before Training is '  , accuracyBefore)                           #Trainingden önceki accuracy


costs = []


for i in range(iterations):
    
    XMini = X[i * Batch_Size:(i + 1)*Batch_Size]     #X ve y için index güncellemesi 0-10, 10-20, 20-30... şeklinde
    yMini = y[i * Batch_Size:(i + 1)*Batch_Size]

    for i1 in range(XMini.shape[0]):

        output_pred = np.dot(XMini[i1], weights)
        errors = yMini[i1] - output_pred   
        weights += (learn_rate * XMini[i1].T.dot(errors))/Batch_Size
        cost = (errors**2).sum() / 2.0
        costs.append(cost)      
          

plt.plot(range(1, len(costs) + 1), costs)
plt.title("Adaline")
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

accuracyMiniBatch = np.sum(y == np.where(np.dot(X, weights) >= 0.0, 1, -1)) / len(y)
print('Accuracy After Training is '  , accuracyMiniBatch)

### Q4 için mantığı kavradığıma inanıyorum fakat atladığım bir şey var çözemediğim, bu yüzden output ve accuracy olarak sonucumun doğruluğundan emin değilim

## Q5 Lineer Regression and Comparing the accuracy between the Adaline

In [ ]:
from sklearn.linear_model import LinearRegression

X=np.array(X_Global)
y= np.array(S)

regressor = LinearRegression()
regressor.fit(X, y)
predictions = regressor.predict(X)
accuracyLinear = np.sum(y == np.where(predictions >= 0.0, 1, -1)) / len(y)

print("Lineer classfication accuracy", accuracyLinear)

In [ ]:
accuracyLinear
accuracySequential
accuracyMiniBatch
accuracyBatch 
print("Lineer classfication accuracy", accuracyLinear)
print("Sequential accuracy", accuracySequential)
print("Batch  accuracy", accuracyBatch )
print("MiniBatch accuracy", accuracyMiniBatch) #Mini batch için emin değilim sonuçtan